# Tensor Flow Notebook
## W207 Final Project
### T. P. Goter
### July 15, 2019

This workbook is used to create convolutional neural nets for facial keypoint detection on CPUs (not TPUs).

In [45]:
# Import needed packages
from __future__ import absolute_import, division, print_function, unicode_literals
import tensorflow as tf
import pandas as pd
import numpy as np
import time
from tensorflow.keras import models, layers, callbacks
from tensorflow.keras import optimizers, metrics
from matplotlib import pyplot as plt
%matplotlib inline

In [44]:
tf.__version__
tf.keras.backend.clear_session()

## Data Loading

- Load in the pickle file that was created as part of the EDA in DataExploration.ipynb. 
- This dataset has the NaNs removed and a few mislabeled images removed as well. 
- As such there is only limited training and development data to use. 
- The image data has already been normalized to [0,1]

In [46]:
# Initialize Random Seed for reproducibility
np.random.seed(13)

# Load the dataframe from the pickle file
df_nostache_nonan = pd.read_pickle("df_nostache_nonan.pkl")

# Grab the last column - that is our image data for X matrix
X = df_nostache_nonan.iloc[:, -1]

# Convert from a series of arrays to an NDarray
X = np.array([x.reshape(96,96,1) for x in X])

# Grab the keypoints and stick into our y-variable
y = np.array(df_nostache_nonan.iloc[:,:-1])
# y = (y - 48) / 48  # scale target coordinates to [-1, 1]

In [48]:
X.shape

(2140, 96, 96, 1)

## Baseline Neural Net
CNNs are combinations of convolution layers, pooling layers and dropout layers followed by one or two fully connected layers once the number of dimensions has been sufficiently reduced.

1. Let's include just one hidden layer and one output layer
2. The input layer will reduce our flattened 96x96 matrix (i.e., 9216 in length) to a predetermined number of hidden units
3. We will then run sensitivities to # of hidden units, activation, optimizer, and learning rate
4. We will judge our model based on RMSE error and run time.

In [53]:
def create_cnn_model():
    '''
    Simple function that retruns a keras cnn model 
    '''
    cnn_model = tf.keras.models.Sequential()
    cnn_model.add(tf.keras.layers.InputLayer(input_shape=(96, 96, 1)))
    cnn_model.add(tf.keras.layers.Conv2D(32, (3, 3), padding='valid', activation='relu'))
    cnn_model.add(tf.keras.layers.MaxPooling2D(pool_size=(2, 2)))
    cnn_model.add(tf.keras.layers.Conv2D(64, (2, 2), padding='valid', activation='relu'))
    cnn_model.add(tf.keras.layers.MaxPooling2D(pool_size=(2, 2)))
    cnn_model.add(tf.keras.layers.Conv2D(128, (2, 2), padding='valid', activation='relu'))
    cnn_model.add(tf.keras.layers.MaxPooling2D(pool_size=(2, 2)))
    cnn_model.add(tf.keras.layers.Flatten())
    cnn_model.add(tf.keras.layers.Dense(500))
    cnn_model.add(tf.keras.layers.Activation('relu'))
    cnn_model.add(tf.keras.layers.Dense(500))
    cnn_model.add(tf.keras.layers.Activation('relu'))
    cnn_model.add(tf.keras.layers.Dense(30))
    cnn_model.add(tf.keras.layers.Activation('linear'))

    print(50*"=")
    print(cnn_model.summary())
    print(50*"=")
  
    return cnn_model
   
class TimeHistory(callbacks.Callback):
    def on_train_begin(self, logs={}):
        self.times = []

    def on_epoch_begin(self, batch, logs={}):
        self.epoch_time_start = time.time()

    def on_epoch_end(self, batch, logs={}):
        self.times.append(time.time() - self.epoch_time_start)

#### Define our optimizers for our initial study

In [54]:
adam = optimizers.Adam(lr=0.001, beta_1=0.9, beta_2=0.999)
sgd = optimizers.SGD(lr=0.01, momentum=0.9, decay=0.0, nesterov=True)
nadam = optimizers.Nadam(lr=0.002, beta_1=0.9, beta_2=0.999, epsilon=None, schedule_decay=0.004)
adagrad = optimizers.Adagrad(lr=0.01, epsilon=None, decay=0.0)

opt_list = {'adam':adam, 'sgd':sgd, 'nadam':nadam, 'adagrad':adagrad}

In [60]:
model.summary()

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_3 (Conv2D)            (None, 94, 94, 32)        320       
_________________________________________________________________
max_pooling2d_3 (MaxPooling2 (None, 47, 47, 32)        0         
_________________________________________________________________
conv2d_4 (Conv2D)            (None, 46, 46, 64)        8256      
_________________________________________________________________
max_pooling2d_4 (MaxPooling2 (None, 23, 23, 64)        0         
_________________________________________________________________
conv2d_5 (Conv2D)            (None, 22, 22, 128)       32896     
_________________________________________________________________
max_pooling2d_5 (MaxPooling2 (None, 11, 11, 128)       0         
_________________________________________________________________
flatten_1 (Flatten)          (None, 15488)            

#### Set up our first DOE

In [61]:
time_callback = TimeHistory()
cnn_base_df = pd.DataFrame()
for opt_name, opt in opt_list.items():
    model = create_cnn_model()
    model.compile(
          optimizer=opt_list['adam'],
          loss='mean_squared_error',
          metrics=['mean_squared_error'])
    history = model.fit(
        X.astype(np.float32), y.astype(np.float32),
        epochs=200,
        validation_split=0.15, callbacks=[time_callback])
    times = time_callback.times
    
    # Convert to dataframe
    hist = pd.DataFrame(history.history)
    hist['epoch'] = history.epoch
    hist['RMSE'] = np.sqrt(hist.mean_squared_error)
    hist['val_RMSE'] = np.sqrt(hist.val_mean_squared_error)
    hist['times'] = times
    hist['layers'] = 3
    hist['pooling'] = 'yes'
    hist['fc_layer'] = 500
    hist['activation'] = 'relu'
    hist['optimizer'] = opt_name
    hist['lrate'] = opt.get_config()['learning_rate']
    
    # Keep concatenating to dataframe
    cnn_base_df = pd.concat([cnn_base_df,hist])

    # Re-pickle after every model to retain progress
    cnn_base_df.to_pickle("OutputData/cnn_base_df.pkl")

    # Save models.
    filename = "cnn_model_{}".format( opt_name)
    model.save("Models/"+filename+".h5")
    

Model: "sequential_2"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_6 (Conv2D)            (None, 94, 94, 32)        320       
_________________________________________________________________
max_pooling2d_6 (MaxPooling2 (None, 47, 47, 32)        0         
_________________________________________________________________
conv2d_7 (Conv2D)            (None, 46, 46, 64)        8256      
_________________________________________________________________
max_pooling2d_7 (MaxPooling2 (None, 23, 23, 64)        0         
_________________________________________________________________
conv2d_8 (Conv2D)            (None, 22, 22, 128)       32896     
_________________________________________________________________
max_pooling2d_8 (MaxPooling2 (None, 11, 11, 128)       0         
_________________________________________________________________
flatten_2 (Flatten)          (None, 15488)            

1819/1819 [==============================] - 10s 5ms/sample - loss: 4.2366 - mean_squared_error: 4.2366 - val_loss: 4.6964 - val_mean_squared_error: 4.6964
Epoch 37/200
1819/1819 [==============================] - 10s 5ms/sample - loss: 4.1095 - mean_squared_error: 4.1095 - val_loss: 4.3056 - val_mean_squared_error: 4.3056
Epoch 38/200
1819/1819 [==============================] - 10s 5ms/sample - loss: 4.1825 - mean_squared_error: 4.1825 - val_loss: 4.2648 - val_mean_squared_error: 4.2648
Epoch 39/200
1819/1819 [==============================] - 10s 5ms/sample - loss: 3.8935 - mean_squared_error: 3.8935 - val_loss: 4.2980 - val_mean_squared_error: 4.2980
Epoch 40/200
1819/1819 [==============================] - 10s 5ms/sample - loss: 4.0959 - mean_squared_error: 4.0959 - val_loss: 4.1418 - val_mean_squared_error: 4.1418
Epoch 41/200
1819/1819 [==============================] - 10s 5ms/sample - loss: 3.7454 - mean_squared_error: 3.7454 - val_loss: 4.1137 - val_mean_squared_error: 4.1137

1819/1819 [==============================] - 10s 5ms/sample - loss: 1.7629 - mean_squared_error: 1.7629 - val_loss: 2.8327 - val_mean_squared_error: 2.8327
Epoch 85/200
1819/1819 [==============================] - 10s 5ms/sample - loss: 1.7365 - mean_squared_error: 1.7365 - val_loss: 2.8916 - val_mean_squared_error: 2.8916
Epoch 86/200
1819/1819 [==============================] - 10s 5ms/sample - loss: 1.6457 - mean_squared_error: 1.6457 - val_loss: 2.8114 - val_mean_squared_error: 2.8114
Epoch 87/200
1819/1819 [==============================] - 10s 5ms/sample - loss: 1.6751 - mean_squared_error: 1.6751 - val_loss: 2.7579 - val_mean_squared_error: 2.7579
Epoch 88/200
1819/1819 [==============================] - 10s 5ms/sample - loss: 1.6433 - mean_squared_error: 1.6433 - val_loss: 2.8155 - val_mean_squared_error: 2.8155
Epoch 89/200
1819/1819 [==============================] - 10s 5ms/sample - loss: 1.6258 - mean_squared_error: 1.6258 - val_loss: 2.9246 - val_mean_squared_error: 2.9246

1819/1819 [==============================] - 10s 5ms/sample - loss: 0.9025 - mean_squared_error: 0.9025 - val_loss: 2.8500 - val_mean_squared_error: 2.8500
Epoch 133/200
1819/1819 [==============================] - 9s 5ms/sample - loss: 0.7754 - mean_squared_error: 0.7754 - val_loss: 2.7243 - val_mean_squared_error: 2.7243
Epoch 134/200
1819/1819 [==============================] - 9s 5ms/sample - loss: 0.8464 - mean_squared_error: 0.8464 - val_loss: 2.6734 - val_mean_squared_error: 2.6734
Epoch 135/200
1819/1819 [==============================] - 9s 5ms/sample - loss: 0.8815 - mean_squared_error: 0.8815 - val_loss: 2.6399 - val_mean_squared_error: 2.6399
Epoch 136/200
1819/1819 [==============================] - 9s 5ms/sample - loss: 0.7124 - mean_squared_error: 0.7124 - val_loss: 2.6558 - val_mean_squared_error: 2.6558
Epoch 137/200
1819/1819 [==============================] - 9s 5ms/sample - loss: 0.7539 - mean_squared_error: 0.7539 - val_loss: 2.7712 - val_mean_squared_error: 2.7712

1819/1819 [==============================] - 10s 5ms/sample - loss: 0.4652 - mean_squared_error: 0.4652 - val_loss: 2.8808 - val_mean_squared_error: 2.8808
Epoch 181/200
1819/1819 [==============================] - 10s 5ms/sample - loss: 0.4906 - mean_squared_error: 0.4906 - val_loss: 3.0376 - val_mean_squared_error: 3.0376
Epoch 182/200
1819/1819 [==============================] - 10s 5ms/sample - loss: 0.8421 - mean_squared_error: 0.8421 - val_loss: 3.1382 - val_mean_squared_error: 3.1382
Epoch 183/200
1819/1819 [==============================] - 10s 5ms/sample - loss: 0.6145 - mean_squared_error: 0.6145 - val_loss: 2.7591 - val_mean_squared_error: 2.7591
Epoch 184/200
1819/1819 [==============================] - 9s 5ms/sample - loss: 0.4124 - mean_squared_error: 0.4124 - val_loss: 2.7161 - val_mean_squared_error: 2.7161
Epoch 185/200
1819/1819 [==============================] - 10s 5ms/sample - loss: 0.3859 - mean_squared_error: 0.3859 - val_loss: 2.7095 - val_mean_squared_error: 2.

NameError: name 'hidden_unit' is not defined

In [17]:
history.history

{'loss': [301.0397809465355,
  300.9012669609955,
  300.8729474202429,
  300.8986479353944,
  300.8937921172777,
  300.9504336650192,
  300.88579441466914,
  300.8508411174688,
  300.84935885907004,
  300.89306451043564,
  300.8886131215318,
  301.07204356306266,
  301.0099840463027,
  301.21548827655243,
  300.99505954132167,
  300.94686564172343,
  300.8758025670065,
  300.88346840143856,
  300.90661314072486,
  300.9949002090295,
  301.05165486810233,
  300.97625205620363,
  300.954881513951,
  300.9295923401852,
  300.8546888153261,
  300.8519757707542,
  300.86015919606194,
  300.8660421764674,
  300.8818669080603,
  300.92487491651707,
  301.22669745513934,
  301.12876500344134,
  300.89463105707654,
  300.86598989896686,
  300.86747727438666,
  300.8321356707841,
  300.86906863438554,
  300.8664869546497,
  300.8651480399494,
  300.86164417235386,
  300.83562084866986,
  300.9438621013488,
  301.00247746027713,
  300.8665602538794,
  300.8337125185756,
  300.8587711747365,
  300